In [1]:
from loadTDMS import *
from fft_test import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns

In [2]:
#　移動平均のための np_array作成
x = np.linspace(-1,1,100)
v = np.exp(-x**2 / 2) / np.sqrt(2 * np.pi)
v = v / np.sum(v)
v = np.sort(v)[::-1]

In [3]:
# FFT条件
dt = 1/10**6
split_t_r = 0.1 # 1つの枠で全体のどの割合のデータを分析するか。
overlap = 0.5  # オーバーラップ率
window_F = "hamming"  # 窓関数選択: hanning, hamming, blackman
y_label = "amplitude"
y_unit = "V"

In [4]:
# FFTするpathのリスト
# ex_path = [3,61,100,150,164]
ex_path = [2,4]

In [5]:
def execute_fft(num,start,end,output_FN,title):
    load_path = rf"Z:\100 Raw_data\20221027---切削実験\AE\Cutting\AE_{num}.tdms"
    d = load_tdms(load_path, ch_num=3)
    d = d[0][start:end]
    
    N = len(d)
    t = np.linspace(0, N*dt, N)  # 周波数軸　linspace(開始,終了,分割数)
   
    fq_, amp_ = FFT_main(t, d, dt, split_t_r, overlap, window_F)
    plot_FFT(fq_, amp_, output_FN, y_label, y_unit, title)

In [7]:
for num in tqdm(ex_path):
    
    #　データ読み込み
    cf = np.load(rf"Z:\200 Produced_data\20221027\CF\{num}.npy")
    
    #　負の値を0に
    cf[cf < 0] = 0
    
    #　移動平均でノイズ除去
    cf_m_avg = np.convolve(cf,v, mode='valid')
    cf_m_avg[cf_m_avg < 5] = 0

    
    #　0であるところのstartとendを特定する.高速ver
    cf_w = np.where(cf_m_avg != 0)[0]
    cf_wd = cf_w[1:] - cf_w[:-1]
    cf_wdd = np.where(cf_wd > 1600)[0]
    cf_wdd = cf_wdd + 1
    
    
    
    for i in range(len(cf_wdd)-1 - 1050):
        start = cf_w[cf_wdd[i]]*3
        end = cf_w[cf_wdd[i+1]]*3
#         print(f"{start}:{end}")
        output_FN = rf"Z:\200 Produced_data\20221027\CF\menseki\fig\100&1600\AE\{num}_{i}"
        title = str(num) + "_" + str(i)
        execute_fft(num,start,end,output_FN,title)

100%|██████████| 2/2 [01:19<00:00, 39.64s/it]
